# 作业6
华东师范大学 | 统计学院  | 作业 |
-|-------|-
课程名称：自然语言处理与实践 | 指导教师：杨燕 | 作业成绩：
专业： 教育信息技术 | 作业编号： 6 | 截止日期：2024年5月28日
年级： | 姓名：   | 学号： |

## 任务描述
给定公告数据集（data/train.jsonl，数据量10w+，类别数262）
1. 构建tf-idf矩阵（类别作为文档）；利用tf-idf矩阵对测试集分类（data/test.jsonl，数据量1w+），计算各类的准确度
2. 基于tf-idf矩阵完成每个公告类别特征选取，使用word2vec模型进行分类，完善word2vec.py
3. 基于tf-idf向量完成多元分类（可以使用sklearn 库分类器）

提示：
1. 预处理：去除停用词、公司简称全称股票代码等；
2. 分词：可以使用jieba工具，考虑有无更好的分词方法，自定义分词词表；
3. 数据量太大处理过程慢，使用word2vec每个类别sample一部分测试数据


其他文件：
data/stop_words.txt 停用词表
data/stock_df.csv  股票代码、简称、全称

In [1]:
import gensim
import json
import numpy as np
import jsonlines
import re
import tqdm
import jieba
import pandas as pd
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression


In [2]:
conda install jsonlines

Retrieving notices: ...working... done
Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 23.7.4
  latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=24.5.0



## Package Plan ##

  environment location: /Users/yishi/anaconda3/envs/yshi

  added / updated specs:
    - jsonlines


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-3.0.13             |       h1a28f6b_2         4.3 MB
    ------------------------------------------------------------
                                           Total:         4.3 MB

The following packages will be UPDATED:

  openssl                                 3.0.13-h1a28f6b_1 --> 3.0.13-h1a28f6b_2 



                                                           

## 数据准备
1.加载自定义词表
2.定义预处理函数
3.得到训练集的词频矩阵df

In [2]:
# 自定义词表
user_dict = './data/user_dict.txt'
# 停用词表
stopwords = [line.strip() for line in open('./data/stop_words.txt', encoding='utf-8').readlines()]
# 扩充停用词
stock_list = pd.read_csv('./data/stock_df.csv')
stopwords.extend(stock_list['name'].to_list())
stopwords.extend(stock_list['fullname'].to_list())
print(len(stopwords))
stopwords = set(stopwords)

12793


In [3]:
def preprocess_and_tokenize(data, stopwords):
    # todo
    texts = []
    labels = []
    
    for entry in data:
        text = entry['title']
        label = entry['type']
        
        # 使用 jieba 进行分词
        words = list(jieba.cut(text))
        # 过滤停用词和长度小于2的词汇
        # 指定抽取的词的数量
        
        
        filtered_words = [word for word in words if word not in stopwords and len(word) > 1]
        
        # 将处理过的文本加入列表
        texts.append(filtered_words)
        # 将标签加入列表
        labels.append(label)
    return texts, labels

In [8]:
# 统计每个公告类别下词频
notices = {}
# 各类公告数量
notices_num = {}
# 读取数据
with open('./data/train.jsonl', 'r', encoding='utf-8') as reader:
    lines = reader.readlines()
data = []
for line in lines:
    data.append(json.loads(line))
print(len(data))

102501


In [9]:
# 预处理：去除非中文字符，分词，去停用词
texts, labels = preprocess_and_tokenize(data,stopwords)


# 并计算每个公告类别的文档数量
# 计算每个公告类别的词频数和文档数量
for label, text in zip(labels, texts):
    if label not in notices_num:
        notices_num[label] = []  # 确保标签的默认值是空列表
    notices_num[label].append(text)
    # notices[label].update(text)
        
print(len(notices_num))


262


In [44]:
total_words = sum(len(sublist) for sublist in notices_num['其他'])
print(total_words)

16726


In [12]:
from collections import Counter
# 并计算每个公告类别的词频数
for key, value in notices_num.items():
    flat_list = [word for sublist in value for word in sublist]
    word_count = Counter(flat_list)
    #if key not in notices:
        #notices[key]={}
    notices[key] = word_count

# notices转换df
df = pd.DataFrame.from_dict(notices)
df.fillna(0)

,其他,股东大会决议公告,高管人员任职变动,监事会决议公告,半年度报告全文,专项说明/独立意见,关联交易,借贷,调研活动,独立董事候选人声明,...,询价转让核查报告/意见,询价转让计划书,科创板并购重组问询与回复,科创板发行上市发行保荐书,科创板发行上市招股说明书注册稿,科创板发行上市问询与回复,科创板发行上市上市保荐书,科创板发行上市审计报告,科创板发行上市法律意见书,科创板发行上市招股说明书上会稿
阳光,2.0,0.0,0.0,2.0,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
电源,2.0,1.0,0.0,0.0,1.0,1.0,3.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
子公司,107.0,0.0,0.0,0.0,0.0,4.0,59.0,153.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
提供,26.0,0.0,0.0,0.0,0.0,3.0,4.0,48.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
担保,43.0,0.0,0.0,0.0,0.0,65.0,2.0,7.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
对科,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
计划书,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
大嘉,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
之七,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


## 分类方法1：计算词的tf-idf值

![local](./images/img1.png)

hints:
1.将训练集按类别统计公告标题的词频并计算TF矩阵
2.统计在训练集中出现的所有词words出现的公告类别数，计算IDF向量
3.计算TF-IDF矩阵
4.测试：输入一个句子完成预处理后，按公告类别统计在words中出现的词的tf-idf权重和
4.预测：将tf-idf和最大的值作为预测类别


In [16]:
# 计算tf
type_tf = {}
for key, value in notices.items():
    # 计算总词数
    total_words = sum(value.values())
    # 计算TF：每个词的频率除以总词数
    tf = {word: count / total_words for word, count in value.items()}
    type_tf[key]=tf
print(type_tf['其他'])


{'阳光': 0.00011957431543704412, '电源': 0.00011957431543704412, '子公司': 0.00639722587588186, '提供': 0.0015544661006815735, '担保': 0.0025708477818964484, '额度': 0.0011957431543704413, '预计': 0.0005380844194666986, '进展': 0.0034078679899557573, '公告': 0.04956355374865479, '上交所': 0.0005380844194666986, '新疆': 0.0005978715771852207, '亿路': 5.978715771852206e-05, '万源': 5.978715771852206e-05, '实业': 0.0008968073657778309, '控股': 0.003886165251703934, '股份': 0.04597632428554346, '有限公司': 0.04663398302044721, '控制': 0.0003587229463111324, '黄伟及': 5.978715771852206e-05, '责任人': 0.00041851010402965444, '纪律处分': 0.0004782972617481765, '盖世': 0.00041851010402965444, '食品': 0.0006576587349037427, '上市': 0.006696161664474471, '保荐': 0.0031687193590816693, '申报': 0.010283391127585794, '财务报表': 0.0034078679899557573, '审计报告': 0.004603611144326199, '江苏': 0.0011957431543704413, '东星': 0.0004782972617481765, '智慧': 0.0005978715771852207, '医疗': 0.001016381681214875, '科技股份': 0.009087647973215353, '东风': 0.0007772330503407868, '科技': 0.0

In [21]:
title_typenum={}
for index, row in df.iterrows():
    non_nan_count = row.notna().sum()
    title_typenum[index] = non_nan_count
print(title_typenum)

{'阳光': 72, '电源': 57, '子公司': 72, '提供': 25, '担保': 22, '额度': 16, '预计': 18, '进展': 66, '公告': 171, '上交所': 9, '新疆': 131, '亿路': 5, '万源': 5, '实业': 146, '控股': 166, '股份': 255, '有限公司': 242, '控制': 75, '黄伟及': 2, '责任人': 7, '纪律处分': 5, '盖世': 24, '食品': 163, '上市': 112, '保荐': 39, '申报': 43, '财务报表': 17, '审计报告': 20, '江苏': 205, '东星': 26, '智慧': 88, '医疗': 197, '科技股份': 234, '东风': 37, '科技': 245, '电子': 221, '特定': 92, '对象': 85, '配股': 15, '预案': 31, '相关': 109, '文件': 33, '修订': 53, '情况': 76, '徐工': 23, '机械': 119, '集团': 178, '工程机械': 11, '吸收': 21, '合并': 36, '关联': 68, '交易': 80, '报告书': 32, '草案': 39, '修订稿': 55, '摘要': 24, '博众': 26, '精工': 158, '688097': 16, '融资': 17, '风险': 36, '提示': 28, '给予': 4, '云南': 108, '药业': 215, '责任': 4, '主体': 19, '本次': 29, '资产重组': 35, '涉及': 34, '购买': 57, '资产': 71, '一期': 20, '财务报告': 20, '广东': 179, '智能': 201, '装备': 154, '集团股份': 213, '终止': 53, '帝科': 26, '审核': 37, '华泰': 100, '联合': 121, '证券': 180, '有限责任': 61, '公司': 184, '发行': 124, '支付现金': 33, '募集': 62, '配套': 37, '资金': 66, '独立': 57, '财务顾问': 21, '报告': 82, '会稿':

In [23]:
import math
# 计算idf
idf = {word: math.log((262 + 1) / (count + 1)) + 1 for word, count in title_typenum.items()}
print(idf)



{'阳光': 2.2816945910293738, '电源': 2.511711021631345, '子公司': 2.2816945910293738, '提供': 3.3140574941562826, '担保': 3.436659816248615, '额度': 3.7389406881215486, '预计': 3.627715053011324, '进展': 2.3674614127867986, '公告': 1.4246595553643115, '上交所': 4.269568939183719, '新疆': 1.6893521095913937, '亿路': 4.780394562949709, '万源': 4.780394562949709, '实业': 1.5817214453990283, '控股': 1.4541602197610095, '股份': 1.026976587698202, '有限公司': 1.0790925888372163, '控制': 2.2414206918914337, '黄伟及': 5.473541743509655, '责任人': 4.4927124904979285, '纪律处分': 4.780394562949709, '盖世': 3.3532782073095637, '食品': 1.4722876043535662, '上市': 1.844766213465424, '保荐': 2.883274578063828, '申报': 2.7879643982595033, '财务报表': 3.6817822742815998, '审计报告': 3.5276315944543417, '江苏': 1.2442778633881835, '东星': 3.2763171661734356, '智慧': 2.0835176624456246, '医疗': 1.2838870014832293, '科技股份': 1.1125685180336056, '东风': 2.934567872451379, '科技': 1.0668224962454018, '电子': 1.169476650305485, '特定': 2.0395545390245085, '对象': 2.117806735924257, '配股': 3.799

In [35]:
# 计算tf-idf
# 计算TF-IDF
tf_idf = {}
for key, tf in type_tf.items():
    tf_idf[key] = {word: type_tf[key][word] * idf[word] for word in tf }

# 输出tf-idf结果
print(tf_idf['其他'])


{'阳光': 0.00027283206875874374, '电源': 0.00030033612598724685, '子公司': 0.014596515678592789, '提供': 0.005151590030375663, '担保': 0.008835129265735408, '额度': 0.004470812732418449, '预计': 0.0019520169482902018, '进展': 0.008067995966091564, '公告': 0.07061119044583368, '上交所': 0.002297388524013719, '新疆': 0.0010100156101825863, '亿路': 0.0002858062036918396, '万源': 0.0002858062036918396, '实业': 0.0014184994428426056, '控股': 0.005651106916445392, '股份': 0.0472166086296734, '有限公司': 0.050322385465325165, '控制': 0.0008040490345180319, '黄伟及': 0.00032724750349812597, '责任人': 0.0018802455717736159, '纪律处分': 0.0022864496295347168, '盖世': 0.0014033808113814987, '食品': 0.0009682628032936283, '上市': 0.0123528527985249, '保荐': 0.009136287973058883, '申报': 0.02866972835708684, '财务报表': 0.012547027958510771, '审计报告': 0.016239844121307207, '江苏': 0.0014878367372810996, '东星': 0.0015670535291992995, '智慧': 0.0012456759909396297, '医疗': 0.0013049192290574495, '科技股份': 0.010110631037971305, '东风': 0.002280843138937458, '科技': 0.00637822848

In [41]:
print(sum(notices['其他'].values()))

16726


In [46]:
# 对每个类别的TF-IDF进行阈值过滤
filtered_tf_idf = {}
for category, words_dict in tf_idf.items():
    # 应用阈值，过滤每个词的TF-IDF
    filtered_tf_idf[category] = {word: tfidf if tfidf >= 0.01 else 0 for word, tfidf in words_dict.items()}

# 输出特定类别（例如'其他'）的TF-IDF结果
print(filtered_tf_idf['其他'])

{'阳光': 0, '电源': 0, '子公司': 0.014596515678592789, '提供': 0, '担保': 0, '额度': 0, '预计': 0, '进展': 0, '公告': 0.07061119044583368, '上交所': 0, '新疆': 0, '亿路': 0, '万源': 0, '实业': 0, '控股': 0, '股份': 0.0472166086296734, '有限公司': 0.050322385465325165, '控制': 0, '黄伟及': 0, '责任人': 0, '纪律处分': 0, '盖世': 0, '食品': 0, '上市': 0.0123528527985249, '保荐': 0, '申报': 0.02866972835708684, '财务报表': 0.012547027958510771, '审计报告': 0.016239844121307207, '江苏': 0, '东星': 0, '智慧': 0, '医疗': 0, '科技股份': 0.010110631037971305, '东风': 0, '科技': 0, '电子': 0, '特定': 0.010608707694316169, '对象': 0, '配股': 0.01817321683576699, '预案': 0, '相关': 0, '文件': 0, '修订': 0, '情况': 0, '徐工': 0, '机械': 0, '集团': 0, '工程机械': 0, '吸收': 0, '合并': 0, '关联': 0, '交易': 0, '报告书': 0, '草案': 0, '修订稿': 0, '摘要': 0, '博众': 0, '精工': 0, '688097': 0, '融资': 0, '风险': 0, '提示': 0, '给予': 0, '云南': 0, '药业': 0, '责任': 0, '主体': 0, '本次': 0.011943696579127309, '资产重组': 0.011971693846666917, '涉及': 0.01244527155192492, '购买': 0.020573024630126405, '资产': 0.02717365818522172, '一期': 0, '财务报告': 0, '广东': 0, '智能

In [32]:

# 设置阈值
threshold = 0.1
# 对权重较低的词进行删减,值设置为0
df_weight = tf_idf.copy()
df_weight[df_weight < threshold] = 0
# 输出删减后的词项-文档矩阵
print(df_weight)

TypeError: '<' not supported between instances of 'dict' and 'float'

In [47]:
# 输入word_list基于df_weight完成分类，权重和最大的类别
def classify(word_list):
     # 初始化一个字典来保存每个类别的总权重
    category_scores = {}

    # 遍历每个类别及其权重数据
    for category, weights in filtered_tf_idf.items():
        # 初始化该类别的得分为0
        category_scores[category] = 0
        # 遍历词列表，累加权重得分
        for word in word_list:
            if word in weights:  # 确保词在权重数据中
                category_scores[category] += weights[word]
    
    # 找出得分最高的类别
    max_category = max(category_scores, key=category_scores.get)
    
    return max_category


In [48]:
# 对采样数据集中每个文档完成分类
classification_result ={}
model = gensim.models.Word2Vec.load('./data/word2vec.model')
# 读取数据
with open('./data/train.jsonl', 'r', encoding='utf-8') as reader:
    lines = reader.readlines()
data = []
for line in lines:
    data.append(json.loads(line))
# 预处理：去除非中文字符，分词，去停用词
texts,labels = preprocess_and_tokenize(data,stopwords)
for idx in (tqdm.tqdm(range(len(labels)))):
    text,label = texts[idx],labels[idx]
    if label not in classification_result.keys():
        classification_result[label] = {
            'correct': 0,
            'count': 0
        }
    # 分类
    type_ = classify(text)
    if type_ == label:
        classification_result[label]['correct'] += 1
    classification_result[label]['count'] += 1
    # break
        
# todo 输出总的正确率和各个类别的正确率（报告中对不同方法进行对比）
for key in classification_result.keys():
    print(f'{key} Acc: {classification_result[key]["correct"]/classification_result[key]["count"]}')

100%|████████████████████████████████████████████████████████████████████████| 102501/102501 [00:24<00:00, 4131.71it/s]

其他 Acc: 0.0
股东大会决议公告 Acc: 0.96625
高管人员任职变动 Acc: 0.09
监事会决议公告 Acc: 1.0
半年度报告全文 Acc: 0.995
专项说明/独立意见 Acc: 0.64625
关联交易 Acc: 0.8325
借贷 Acc: 0.61875
调研活动 Acc: 0.76375
独立董事候选人声明 Acc: 0.99125
股东大会资料 Acc: 0.96
召开股东大会通知 Acc: 0.98875
年度报告摘要 Acc: 0.99875
独立董事述职报告 Acc: 1.0
年度报告全文 Acc: 0.95
保荐/核查意见 Acc: 0.0
内部控制报告 Acc: 0.97375
审计机构变更 Acc: 0.99125
审计报告 Acc: 0.8375
分配预案 Acc: 0.4175
董事会决议公告 Acc: 0.99625
一季度报告全文 Acc: 0.9875
法律意见书 Acc: 0.01375
收购出售资产/股权 Acc: 0.1375
分配方案决议公告 Acc: 0.9425
分配方案实施 Acc: 0.99125
年度报告补充公告 Acc: 0.625
半年度财务报告 Acc: 0.95125
资金占用 Acc: 0.965
半年度报告摘要 Acc: 0.99875
三季度报告全文 Acc: 0.99125
独立董事提名人声明 Acc: 1.0
业绩快报 Acc: 1.0
增发预案 Acc: 0.1
提供/对外担保公告 Acc: 0.88
月度经营情况 Acc: 0.6125
召开股东大会提示性公告 Acc: 0.965
公司章程修订 Acc: 0.775
回购报告书 Acc: 1.0
社会责任报告 Acc: 1.0
股东/实际控制人股份增持 Acc: 0.80875
回购预案 Acc: 0.8876712328767123
回购进展情况 Acc: 0.4175
回购实施公告 Acc: 0.24786324786324787
股份质押、冻结 Acc: 0.9825
股本变动 Acc: 0.26625
投资设立公司 Acc: 0.86
获得补贴（资助） Acc: 0.8475
募集资金使用情况报告 Acc: 0.8875
处罚 Acc: 0.9486166007905138
管理办法/制度 Acc: 0.17

## 基于word2vec 做相似度计算

![本地](./images/img2.png)

Hints:
1.基于方法1算出的tf-idf矩阵，选出每个公告类型tf-idf top10的词作为类别特征词
2.测试：输入句子完成预处理后，确定与某一类公告类别的相似度（计算sentence_list中每个词与特征词的相似度矩阵，取平均）
3.预测：类别相似度最大的类别作为预测类别
建议
请再运行一次word2vec.py文件
在测试集上预测按公告类别分别采样最多5个样例完成测试

In [49]:
# 将每个公告类别词频前10的词作为特征词
words = set()
feature_dic = {}
for type_ in notices.keys():
    top_words  = notices[type_].most_common(10)
     # 只保留词，忽略频率
    feature_words = [word for word, freq in top_words]
    feature_dic[type_] = feature_words
    words.update(feature_words)
print(feature_dic)

{'其他': ['公告', '有限公司', '股份', '报告', '发行', '年度', '资产', '申报', '公司', '公开'], '股东大会决议公告': ['公告', '临时', '股东', '大会决议', '第一次', '第二次', '股份', '有限公司', '第三次', '科技'], '高管人员任职变动': ['公告', '决议', '董事会', '会议', '股份', '监事会', '董事', '公司', '有限公司', '辞职'], '监事会决议公告': ['监事会', '决议', '公告', '会议', '股份', '有限公司', '第三届', '第四届', '第五届', '第二届'], '半年度报告全文': ['报告', '半年度', '股份', '有限公司', '科技', '科技股份', '全文', '集团股份', '集团', '更正'], '专项说明/独立意见': ['独立', '意见', '董事', '事项', '董事会', '相关', '会议', '公司', '股份', '有限公司'], '关联交易': ['关联', '交易', '公告', '日常', '年度', '预计', '股份', '有限公司', '公司', '资产'], '借贷': ['公告', '申请', '银行', '授信额度', '综合', '公司', '年度', '股份', '借款', '子公司'], '调研活动': ['活动', '投资者', '关系', '记录表', '股份', '信息', '调研', '科技', '说明会', '业绩'], '独立董事候选人声明': ['候选人', '声明', '董事', '独立', '股份', '有限公司', '科技', 'ST', '药业', '科技股份'], '股东大会资料': ['股东大会', '会议', '资料', '临时', '有限公司', '股份', '年度', '第一次', '第二次', '材料'], '召开股东大会通知': ['股东大会', '通知', '临时', '第一次', '年度', '股份', '有限公司', '第二次', '公告', '第三次'], '年度报告摘要': ['摘要', '年度报告', '股份', '有限公司', '科技', '更正', '药业', 'ST', '生物', '电子'], '

In [ ]:
# 请再运行一次word2vec文件得到新的word2vec.model文件
model = gensim.models.Word2Vec.load('./data/word2vec.model')

In [ ]:
# tool1:定义一个函数，该函数接受行和列的标签，并返回计算后的值用于计算某一公告类别下特征字典中每个词和输入sentence_list每个词的相似度
def calculate_value(row_label, col_label,model):
    
    return score

In [ ]:
# tool2:use tool1句子和每个公告的特征词的相似度矩阵
def similarity_matrix(df,model):
    
    return sim

In [ ]:
# use tool2定义一个函数使用similarity_matrix 计算模型与每个公告类别的特征词向量的相似度。
def word2vecClassify(word_list,model):
    scores = {}
    for key,values in feature_dic.items():
        df = pd.DataFrame(columns = list(values),index = list(word_list))
        sim = similarity_matrix(df,model)
        scores[key] = sim
    max_type_= max(scores,key=scores.get)
    return max_type_

In [ ]:
# tool3:定义按公告类别对样本采样函数，每个类别下采样5个
def sample_(group):
    
    return sample_numbs

In [ ]:
# use tool3对测试样本采样
with open('./data/test.jsonl', 'r', encoding='utf-8') as reader:
    lines = reader.readlines()
    dt = [json.loads(line) for line in lines]
    dt = pd.DataFrame(dt)
    dt = dt.groupby("type").apply(sample_)


In [ ]:
dt_dict = dt.to_dict(orient ="records")

In [ ]:
# 对采样数据集中每个文档完成分类
classification_result ={}
model = gensim.models.Word2Vec.load('./data/word2vec.model')

# 预处理：去除非中文字符，分词，去停用词
texts,labels = preprocess_and_tokenize(dt_dict,stopwords)
for idx in (tqdm.tqdm(range(len(labels)))):
    text,label = texts[idx],labels[idx]
    if label not in classification_result.keys():
        classification_result[label] = {
            'correct': 0,
            'count': 0
        }
    # 分类
    type_ = word2vecClassify(text,model)
    if type_ == label:
        classification_result[label]['correct'] += 1
    classification_result[label]['count'] += 1
    # break
        
# todo 输出总的正确率和各个类别的正确率（报告中对不同方法进行对比）
for key in classification_result.keys():
    print(f'{key} Acc: {classification_result[key]["correct"]/classification_result[key]["count"]}')

## 基于tf-idf加权的词向量完成多元分类

![本地](./images/img3.png)

Hints:
1.完成对word2vec的封装（定义模型加载，句子分词后转换为特征词向量函数以及考虑加入特殊词<pad><unk>）
2.训练集特征向量提取：特征词向量是将预处理后的每个词计算词向量按维度相加的结果（最后得到一个100维的向量）
3.训练逻辑回归模型
4.测试：输入一个句子完成预处理后，进行特征提取，用训练好的分类器完成预测
测试集的选取同第二个方法

In [ ]:
# 完成对word2vec的封装:
from gensim.models import Word2Vec
class Preprocess:
    def __init__(self, embedding_dim, model_path=None, y=None):
        self.model_path = model_path
        self.embedding_dim = embedding_dim
        self.embedding_matrix = []
        self.word2index = {}
        self.words = []
        self.y = y
        

    def load_word_embedding(self, file):
        # 路径中查询word2vec 或者临时创建 word2vec对输入的句子创建一个字典，同时对每个word embedding
        self.embedding_matrix = []
        self.embedding = Word2Vec.load(file)
        model = self.embedding
        self.word2index = model.wv.key_to_index
        self.words = model.wv.index_to_key
        for i in range(len(model.wv)):  # "Key '23428' not present"
            self.embedding_matrix.append(model.wv[i])
        self.embedding_matrix = np.array(self.embedding_matrix)
        print("loading word2vec model……")
        
        

    def add_embedding(self, word):
        # 在词向量矩阵中加入 <pad\unik> 在load embedding 之后
        # 先随机化一个vector
        vector = np.random.uniform(0, 1, (1, self.embedding_dim))
        words_size = len(self.words)
        self.words.append(word)  # append 让words none
        self.word2index[word] = words_size
        self.embedding_matrix = np.concatenate([self.embedding_matrix, vector], axis=0)

    def make_embedding(self, file=None):
        """
        外部调用 封装了add 控制了load_add的顺序
        """
        if file != None:
            self.load_word_embedding(file)
        elif self.embedding_matrix != []:
            None
        elif self.model_path != None:
            self.load_word_embedding(self.model_path)
        else:
            raise Exception
        self.add_embedding("<PAD>")
        self.add_embedding("<UNK>")
        print("initialize word2vec model……")
        

    def sen2vec(self,sen_list):  # 将sen_list转换为对应的embedding
        
        return vector
p = Preprocess(100,'./data/word2vec.model')
p.make_embedding()




In [ ]:
# 定义特征词向量
def get_feature_vector(words_list,p):
    vector = p.sen2vec(words_list)
    return vector

In [ ]:

# 加载训练集：完成数据预处理和特征提取
with open('./data/train.jsonl', 'r', encoding='utf-8') as reader:
    train_data = [line for line in jsonlines.Reader(reader)]
texts, labels = preprocess_and_tokenize(train_data, stopwords)

# 特征提取

X_train = vectors
Y_train = labels

In [ ]:
# 训练逻辑回归模型



In [ ]:
# 分类并计算每个类别的准确率
# 对采样数据集中每个文档完成分类

# 预处理：去除非中文字符，分词，去停用词
texts,labels = preprocess_and_tokenize(dt_dict,stopwords)
for idx in (tqdm.tqdm(range(len(labels)))):
    text,label = texts[idx],labels[idx]
    if label not in classification_result.keys():
        classification_result[label] = {
            'correct': 0,
            'count': 0
        }
    # 分类
    vector = get_feature_vector(text,p)
    type_ = classifier.predict([vector])[0]
    if type_ == label:
        classification_result[label]['correct'] += 1
    classification_result[label]['count'] += 1
    # break
        
# todo 输出总的正确率和各个类别的正确率（报告中对不同方法进行对比）
for key in classification_result.keys():
    print(f'{key} Acc: {classification_result[key]["correct"]/classification_result[key]["count"]}')